In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [26]:
data1 = pd.read_csv('../prices/prices_round_3_day_0.csv', header = 0, sep=";")
data2 = pd.read_csv('../prices/prices_round_3_day_1.csv', header = 0, sep=";")
data3 = pd.read_csv('../prices/prices_round_3_day_2.csv', header = 0, sep=";")

data = pd.concat([data1,data2, data3])

midprices_column = data.loc[:, "mid_price"]
strawberry_midprices = midprices_column[data["product"] == "STRAWBERRIES"]
chocolate_midprices = midprices_column[data["product"] == "CHOCOLATE"]
roses_midprices = midprices_column[data["product"] == "ROSES"]
gift_basket_midprices = midprices_column[data["product"] == "GIFT_BASKET"]


#s_gb = pd.DataFrame({"STRAWBERRIES":strawberry_midprices, "GIFT_BASKET":gift_basket_midprices})
#c_gb = pd.DataFrame({"CHOCOLATE":chocolate_midprices, "GIFT_BASKET":gift_basket_midprices})
#r_gb = pd.DataFrame({"ROSES":roses_midprices, "GIFT_BASKET":gift_basket_midprices})

strawberry_pchange = strawberry_midprices.pct_change()
chocolate_pchange = chocolate_midprices.pct_change()
roses_pchange = roses_midprices.pct_change()

ys = strawberry_pchange[::4]
ys = ys[1:]
yc = chocolate_pchange[::4]
yc = yc[1:]
yr = roses_pchange[::4]
yr = yr[1:]

strawberry_midprices = strawberry_midprices.to_numpy()
chocolate_midprices = chocolate_midprices.to_numpy()
roses_midprices = roses_midprices.to_numpy()

Xs = strawberry_midprices.reshape(-1,4)
Xs = Xs[:-1]
Xc = chocolate_midprices.reshape(-1,4)
Xc = Xc[:-1]
Xr = roses_midprices.reshape(-1,4)
Xr = Xr[:-1]

#print(strawberry_midprices[:6])
#print(strawberry_pchange[:25])
#print(ys[:5])

In [27]:
#Strawberries
lrs = LinearRegression()
lrs.fit(Xs, ys)

# Coefficients
print("Strawberries Coefficients:", lrs.coef_)
print("Strawberries Intercept:", lrs.intercept_)

predictions = lrs.predict(Xs)
mses = mean_squared_error(ys, predictions)
print("Strawberries Mean squared error:", mses)
print("Strawberries Mean Y squared change:", np.mean(ys ** 2))

#Chocolate
lrc = LinearRegression()
lrc.fit(Xc, yc)

# Coefficients
print("Chocolate Coefficients:", lrc.coef_)
print("Chocolate Intercept:", lrc.intercept_)

predictions = lrc.predict(Xc)
msec = mean_squared_error(yc, predictions)
print("Chocolate Mean squared error:", msec)
print("Chocolate Mean Y squared change:", np.mean(yc ** 2))

#Roses
lrr = LinearRegression()
lrr.fit(Xr, yr)

# Coefficients
print("Roses Coefficients:", lrr.coef_)
print("Roses Intercept:", lrr.intercept_)

predictions = lrr.predict(Xs)
mser = mean_squared_error(yr, predictions)
print("Roses Mean squared error:", mser)
print("Roses Mean Y squared change:", np.mean(yr ** 2))


ValueError: Found input variables with inconsistent numbers of samples: [7500, 7499]

In [ ]:


# Sample Pandas Series
data = [10, 15, 20, 25, 30]
series = pd.Series(data)

# Calculate percentage change
percentage_change = series.pct_change() * 100

print(percentage_change)

0          NaN
1    50.000000
2    33.333333
3    25.000000
4    20.000000
dtype: float64
